# Chicago crimes dataset: Exploratory Data Analysis

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import numpy as np
import glob
from datetime import datetime
#used to display all of the columns
pd.set_option('display.max_columns',100)

%matplotlib inline
% cd crimes

/Users/donovanadams/Desktop/GitHub/DS-3-Deep-Learning/notebooks/crimes


## 1. Data loading and cleaning

### 1a. Data loading

In [2]:
# initial load of dataset
all_csv = glob.glob('*')
for csv in all_csv:
    print(csv)

Chicago_Crimes_2008_to_2011.csv
Chicago_Crimes_2001_to_2004.csv
Chicago_Crimes_2012_to_2017.csv
Chicago_Crimes_2005_to_2007.csv


In [3]:
#loading all files into one csv
def readFilesFolder():
    #reads in the list of file string locations and creates new dfs out of them,it then appends them
    data_frames_list=list()
    for csv in all_csv:
        new_df=pd.read_csv(csv,error_bad_lines=False)
        data_frames_list.append(new_df)
        
    df=pd.concat(data_frames_list)
    return df
starting_df = readFilesFolder()


b'Skipping line 1149094: expected 23 fields, saw 41\n'
b'Skipping line 1513591: expected 23 fields, saw 24\n'
/Users/donovanadams/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
b'Skipping line 533719: expected 23 fields, saw 24\n'


In [4]:
starting_df.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,388,4785,HP610824,10/07/2008 12:39:00 PM,000XX E 75TH ST,0110,HOMICIDE,FIRST DEGREE MURDER,ALLEY,True,False,323,3.0,6.0,69.0,01A,1178207.0,1.85531e+06,2008.0,08/17/2015 03:03:40 PM,41.7583,-87.622451,"(41.758275857, -87.622451031)"
1,835,4786,HP616595,10/09/2008 03:30:00 AM,048XX W POLK ST,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,1533,15.0,24.0,25.0,01A,1144200.0,1.89586e+06,2008.0,08/17/2015 03:03:40 PM,41.8703,-87.746069,"(41.87025207, -87.746069362)"
2,1334,4787,HP616904,10/09/2008 08:35:00 AM,030XX W MANN DR,0110,HOMICIDE,FIRST DEGREE MURDER,PARK PROPERTY,False,False,831,8.0,18.0,66.0,01A,1157314.0,1.85978e+06,2008.0,08/17/2015 03:03:40 PM,41.771,-87.698901,"(41.770990476, -87.698901469)"
3,1907,4788,HP618616,10/10/2008 02:33:00 AM,052XX W CHICAGO AVE,0110,HOMICIDE,FIRST DEGREE MURDER,RESTAURANT,False,False,1524,15.0,37.0,25.0,01A,1141065.0,1.90482e+06,2008.0,08/17/2015 03:03:40 PM,41.8949,-87.757358,"(41.894916924, -87.757358147)"
4,2436,4789,HP619020,10/10/2008 12:50:00 PM,026XX S HOMAN AVE,0110,HOMICIDE,FIRST DEGREE MURDER,GARAGE,False,True,1032,10.0,22.0,30.0,01A,1154123.0,1.8863e+06,2008.0,08/17/2015 03:03:40 PM,41.8438,-87.709893,"(41.843826272, -87.709893465)"


In [5]:
starting_df['Block'].describe()

count              7941282
unique               58776
top       100XX W OHARE ST
freq                 19927
Name: Block, dtype: object

### Converting the blocks fromthe trunicated street numbers and names to just the street name

#### Due to the format it was in, it was easier to trunicate the entire address part consisting od something like this "100XX W" it turns out the address were trunicated for identity

In [6]:
block_from_df =list(map(lambda x: x[8:], starting_df['Block']))

In [7]:
type(block_from_df)
starting_df['Block']=block_from_df

In [8]:
starting_df['Block'].describe()

count      7941282
unique        3592
top       STATE ST
freq        131933
Name: Block, dtype: object

### Converting the date to an actual datetime for ease of use in the rest of the descriptive stats

In [9]:
starting_df['Date'] = pd.to_datetime(starting_df['Date'], format='%m/%d/%Y %H:%M:%S %p')

In [10]:
starting_df['Date'].describe()

count                 7941282
unique                1998692
top       2008-01-01 12:01:00
freq                      565
first     2001-01-01 01:00:00
last      2017-01-18 12:55:00
Name: Date, dtype: object

In [11]:
starting_df.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,388,4785,HP610824,2008-10-07 12:39:00,75TH ST,0110,HOMICIDE,FIRST DEGREE MURDER,ALLEY,True,False,323,3.0,6.0,69.0,01A,1178207.0,1.85531e+06,2008.0,08/17/2015 03:03:40 PM,41.7583,-87.622451,"(41.758275857, -87.622451031)"
1,835,4786,HP616595,2008-10-09 03:30:00,POLK ST,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,1533,15.0,24.0,25.0,01A,1144200.0,1.89586e+06,2008.0,08/17/2015 03:03:40 PM,41.8703,-87.746069,"(41.87025207, -87.746069362)"
2,1334,4787,HP616904,2008-10-09 08:35:00,MANN DR,0110,HOMICIDE,FIRST DEGREE MURDER,PARK PROPERTY,False,False,831,8.0,18.0,66.0,01A,1157314.0,1.85978e+06,2008.0,08/17/2015 03:03:40 PM,41.771,-87.698901,"(41.770990476, -87.698901469)"
3,1907,4788,HP618616,2008-10-10 02:33:00,CHICAGO AVE,0110,HOMICIDE,FIRST DEGREE MURDER,RESTAURANT,False,False,1524,15.0,37.0,25.0,01A,1141065.0,1.90482e+06,2008.0,08/17/2015 03:03:40 PM,41.8949,-87.757358,"(41.894916924, -87.757358147)"
4,2436,4789,HP619020,2008-10-10 12:50:00,HOMAN AVE,0110,HOMICIDE,FIRST DEGREE MURDER,GARAGE,False,True,1032,10.0,22.0,30.0,01A,1154123.0,1.8863e+06,2008.0,08/17/2015 03:03:40 PM,41.8438,-87.709893,"(41.843826272, -87.709893465)"


### Difference between district, ward, community area and beat and why I chose to use beats.
#### A ward is always a legally defined political subdivision. A district may be legally defined,and in this case it is. Community areas are a bit more archaic and were set by social scientists to track demographic changes, this unfortunately has not evolved with the growth and change of the city.  
#### The item that makes the most sense for this is the "beat" which is a small block/ neigborhood or section of town that has a dedicated police man assigned to it. This of this as the police box from England, Japan, or SK

In [12]:
starting_df['Block'].describe()

count      7941282
unique        3592
top       STATE ST
freq        131933
Name: Block, dtype: object

### 1b. Dropping columns with no discernable effect on the stats or that have been picked over for a more preferential label( in this case: ward,District, Unnamed: 0,	ID,	Case Number,FBI Code,Updated On,Latitude,Longitude,X Coordinate,Y Coordinate,IUCR)

### 1c. Size and demonsionality

In [13]:
#total number of rows
print('The total number of rows is {}, and the total number of columns is {}, for a total number of datapoints being {}'.format(len(starting_df),len(starting_df.columns),len(starting_df)*len(starting_df.columns)))

The total number of rows is 7941282, and the total number of columns is 23, for a total number of datapoints being 182649486


### 1c. Checking for NaN values

In [14]:
# By columns
for column in starting_df.columns:
    if any(pd.isna(starting_df[column])):
        print('NA values are in the columns {}. the count is{}'.format(column,pd.isna(starting_df[column]).value_counts()))
    
#Luckily all of that data that I would be using for this set of stats do not use these ( I was originally going to use the location, but ti turns out that police men in this city do not alway but this at the reports so there are numvrous missing values)

NA values are in the columns Case Number. the count isFalse    7941275
True           7
Name: Case Number, dtype: int64
NA values are in the columns Location Description. the count isFalse    7939292
True        1990
Name: Location Description, dtype: int64
NA values are in the columns District. the count isFalse    7941191
True          91
Name: District, dtype: int64
NA values are in the columns Ward. the count isFalse    7241058
True      700224
Name: Ward, dtype: int64
NA values are in the columns Community Area. the count isFalse    7239191
True      702091
Name: Community Area, dtype: int64
NA values are in the columns X Coordinate. the count isFalse    7835709
True      105573
Name: X Coordinate, dtype: int64
NA values are in the columns Y Coordinate. the count isFalse    7835709
True      105573
Name: Y Coordinate, dtype: int64
NA values are in the columns Latitude. the count isFalse    7835709
True      105573
Name: Latitude, dtype: int64
NA values are in the columns Longitude

In [15]:
starting_df=starting_df.drop(['Case Number','Longitude','Y Coordinate','X Coordinate','Ward','District','Case Number','FBI Code','Unnamed: 0','IUCR','Community Area','Updated On','Updated On','ID','Latitude'], axis=1)

,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,Year,Location
0,2008-10-07 12:39:00,75TH ST,HOMICIDE,FIRST DEGREE MURDER,ALLEY,True,False,323,2008.0,"(41.758275857, -87.622451031)"
1,2008-10-09 03:30:00,POLK ST,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,1533,2008.0,"(41.87025207, -87.746069362)"
2,2008-10-09 08:35:00,MANN DR,HOMICIDE,FIRST DEGREE MURDER,PARK PROPERTY,False,False,831,2008.0,"(41.770990476, -87.698901469)"
3,2008-10-10 02:33:00,CHICAGO AVE,HOMICIDE,FIRST DEGREE MURDER,RESTAURANT,False,False,1524,2008.0,"(41.894916924, -87.757358147)"
4,2008-10-10 12:50:00,HOMAN AVE,HOMICIDE,FIRST DEGREE MURDER,GARAGE,False,True,1032,2008.0,"(41.843826272, -87.709893465)"


In [27]:
starting_df['Year'].astype(int)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'map'

In [24]:
starting_df['Year']

0          <map object at 0x1ad1585908>
1          <map object at 0x1ad1585908>
2          <map object at 0x1ad1585908>
3          <map object at 0x1ad1585908>
4          <map object at 0x1ad1585908>
5          <map object at 0x1ad1585908>
6          <map object at 0x1ad1585908>
7          <map object at 0x1ad1585908>
8          <map object at 0x1ad1585908>
9          <map object at 0x1ad1585908>
10         <map object at 0x1ad1585908>
11         <map object at 0x1ad1585908>
12         <map object at 0x1ad1585908>
13         <map object at 0x1ad1585908>
14         <map object at 0x1ad1585908>
15         <map object at 0x1ad1585908>
16         <map object at 0x1ad1585908>
17         <map object at 0x1ad1585908>
18         <map object at 0x1ad1585908>
19         <map object at 0x1ad1585908>
20         <map object at 0x1ad1585908>
21         <map object at 0x1ad1585908>
22         <map object at 0x1ad1585908>
23         <map object at 0x1ad1585908>
24         <map object at 0x1ad1585908>
